# Generating images with MX-Font model from a reference style
In this example we'll generate images with trained MX-Font model from a reference style.
If you want to generate multiple styles, please check using `eval.py` instead of using this example file (because it is much simpler to load the referece styles).

### 1. Loading packages
* First, load the packages used in this code.
* All of the packages are avilable in `pip`.

In [1]:
import json
from pathlib import Path
from PIL import Image

import torch
from sconf import Config
from torchvision import transforms

* These modules are defined in this repository.

In [2]:
import models
from datasets import read_font, render
from utils import save_tensor_to_image

### 2. Build model
* Build and load the trained model.
* `weight_path` : 
    * The location of the trained model weight.

In [4]:
########################################################
#weight_path = "/opt/mxfont/generator.pth"  # path to weight to infer
weight_path ='/opt/final-project-level2-cv-11/src/model/font_generator/mxfont/result/checkpoints/last.pth'
########################################################

cfg = Config("cfgs/eval.yaml", default="cfgs/defaults.yaml")
transform = transforms.Compose(
    [transforms.Resize((128, 128)), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
)
decomposition = json.load(open("data/kor_decomposition.json"))

g_kwargs = cfg.get('g_args', {})
gen = models.Generator(1, cfg.C, 1, **g_kwargs).cuda().eval()
weight = torch.load(weight_path)
if "generator_ema" in weight:
    weight = weight["generator_ema"]
gen.load_state_dict(weight)

<All keys matched successfully>

### 3. Load reference images.
* `ref_path`: 
    * The path of reference font or images.
    * If you are using a ttf file, set this to the location of the ttf file.
    * If you want to use rendered images, set this to the path to the directory which contains the reference images.
* `extension`:
    * If you are using image files, set this to their extension(png, jpg, etc..). 
    * This will be ignored if `use_ttf` is True.
* `batch_size`:
    * The number of images inferred at once.

In [6]:
########################################################
ref_path = "/opt/final-project-level2-cv-11/src/model/font_generator/mxfont/data/images/test"  # Path to the reference images/opt/mxfont/data/images/test/MaShanZheng-Regular
extension = "png"  # Extension of the reference images
batch_size = 3  # The batch size
########################################################

ref_paths = Path(ref_path).glob(f"*.{extension}")
ref_imgs = torch.stack([transform(Image.open(str(p))) for p in ref_paths]).cuda()
ref_batches = torch.split(ref_imgs, batch_size)

### 4. Extract style factors from reference images.

In [7]:
style_facts = {}

for batch in ref_batches:
    style_fact = gen.factorize(gen.encode(batch), 0)
    for k in style_fact:
        style_facts.setdefault(k, []).append(style_fact[k])
        
style_facts = {k: torch.cat(v).mean(0, keepdim=True) for k, v in style_facts.items()}

### 5. Generate the images.
* `gen_chars`: The characters to generate.
* `save_dir`: Path to save the generated images.
* `source_path`: Path to the font file used as the source font.

In [10]:
########################################################
gen_chars = "國"  # Characters to generate
save_dir = Path("./results")  # Directory where you want to save generated images
source_path = "/opt/final-project-level2-cv-11/src/model/font_generator/mxfont/data/ttfs/val/SeoulHangangB.ttf"  # Path to the font file to render the source images
########################################################

save_dir.mkdir(parents=True, exist_ok=True)

source_font = read_font(source_path)
for char in gen_chars:
    source_img = transform(render(source_font, char)).unsqueeze(0).cuda()
    char_facts = gen.factorize(gen.encode(source_img), 1)
    
    gen_feats = gen.defactorize([style_facts, char_facts])
    out = gen.decode(gen_feats).detach().cpu()[0]

    path = save_dir / f"{char}.png"
    save_tensor_to_image(out, path)

In [40]:
import cv2
a = cv2.imread("/opt/mxfont/data/images/test/kor/a-1.png")
b = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
c = cv2.cvtColor(b, cv2.COLOR_BGR2GRAY)

cv2.imwrite('/opt/mxfont/data/images/test/kor/b-1.png', c)

True

In [5]:
from PIL import ImageFilter
from PIL import Image
import matplotlib.pyplot as plt

im = Image.open('/opt/mxfont/results/W.png')

im1 = im.filter(ImageFilter.BLUR)

im2 = im.filter(ImageFilter.MinFilter(3))
im3 = im.filter(ImageFilter.MinFilter) 

plt.imshow(im3)

ImportError: Matplotlib requires numpy>=1.19; you have 1.16.6

In [7]:
import os
import glob
file_list = glob.glob('/opt/final-project-level2-cv-11/src/model/font_generator/mxfont/data/*')
file_list

['/opt/final-project-level2-cv-11/src/model/font_generator/mxfont/data/kor_decomposition.json',
 '/opt/final-project-level2-cv-11/src/model/font_generator/mxfont/data/kor_primals.json',
 '/opt/final-project-level2-cv-11/src/model/font_generator/mxfont/data/ttfs',
 '/opt/final-project-level2-cv-11/src/model/font_generator/mxfont/data/chn_gen.json',
 '/opt/final-project-level2-cv-11/src/model/font_generator/mxfont/data/chn_decomposition.json',
 '/opt/final-project-level2-cv-11/src/model/font_generator/mxfont/data/chn_primals.json',
 '/opt/final-project-level2-cv-11/src/model/font_generator/mxfont/data/kor_gen.json',
 '/opt/final-project-level2-cv-11/src/model/font_generator/mxfont/data/images']

In [14]:
import json
import os

os.environ['HOME'] = "/opt/final-project-level2-cv-11"
with open (os.path.join(os.getenv("HOME"),"src/model/svg2ttf/example.json" ), "r") as f:
                font_json = json.load(f)
                
font_json.keys()

dict_keys(['props', 'glyphs', 'sfnt_names', 'input', 'output', '# vim: set et sw=2 ts=2 sts=2:'])